### 读入tools 生成的possible matchings

In [23]:

# matchings_path = "./dataset/employee.json"
# answers_path = "./dataset/employee_ans.json"
# matchings_path = "./dataset/author.json"
# answers_path = "./dataset/author_ans.json"
matchings_path = "./dataset/purchase.json"
answers_path = "./dataset/purchase_ans.json"
# dataset_name = "author"
dataset_name = "employee"

#LLM 预测准确率
p_w = 0.917
api_use = False
# 每个token 一个
budget = 10 # 单位为 price of one token  
total_turns =20

In [24]:
from fact import FactSet
from llm_api import gpt_check
import numpy as np
import json
import time

with open(matchings_path, "r") as f:
    content = json.load(f)

### 初始化 chatgpt 的 tokenzier，用于估计correspondence 的cost func

In [25]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

### process 统计，factset 需要的数据

In [26]:
def process(content):
    import numpy as np
    c_set = content["correspondence_set"]
    # chatgpt 的tokens 的num list
    len_list = [len(encoding.encode(i[0][1]+i[1][1])) for i in c_set]
    len_list.sort()
    least_len = sum(len_list[-3:])
    matchings = content["matchings"]
    Views = []
    for match in matchings:
        view = []
        for c in c_set:
            if c in match:
                view.append(1)
            else:
                view.append(0)
        Views.append(view)
    return np.array(Views), np.array(content["prob_all"]), np.array(len_list), least_len, sum(len_list), c_set

### 处理后生成 facts 

In [27]:
facts, prob, len_list, least_len, sum_c, c_set = process(content)
print("max_cost: ", least_len, f"note that budget of each round must > {least_len} and < {sum_c}:")
ex_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)
random_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)
brute_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)
heuristic_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)

max_cost:  22 note that budget of each round must > 22 and < 111:


## selector to select correspondence

### 初始化 selector

In [28]:
from query import QuerySelector, BaseQuerySelector, GreedyQuerySelector,RandomQuerySelector, HeuristicQuerySelector
 # 对应fact1, 3是0.8, 0.
query_selector = GreedyQuerySelector()
# selection_idxes, sub_facts, h = query_selector.select(ex_fact, 2, accuracy, cost_func=2)
random_selector = RandomQuerySelector()
base_selector = BaseQuerySelector()
h_selector = HeuristicQuerySelector()

## LLM答案dict
## 由于数量问题，模拟实验，从web chatgpt端，得到所有的correpspondence 验证答案

In [29]:
import json
with open(answers_path, "r") as f:
    ans_list = json.load(f)

### 参数设置

In [30]:
c_len = ex_fact.num_fact()
# 计算p(A_T) 和 P(A_T|v) 需要的LLM array:acc 
acc = np.array([[p_w for i in range(c_len)]])

In [31]:
def post_p_caculate(prior_p, p_a_v, p_a):
    return prior_p*p_a_v / p_a

####  our近似算法

In [32]:
import numpy as np
import time
approx_h_list=[ex_fact.compute_entropy()]

cost_sum = 0
turns = total_turns 
start = time.time()
while turns>0:
    selection_idxes, sub_facts, h = query_selector.select(ex_fact, budget, acc, cost_func=2)
    if api_use:
         ans = [1 if gpt_check(ix_r, c_set)=="yes" else 0 for ix_r in selection_idxes]
    else:
        ans = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in selection_idxes]
    p_a,p_a_v = ex_fact.compute_ans_p(ans, selection_idxes, acc)
    p_post = ex_fact.get_prior_p()*p_a_v/p_a
    ex_fact.set_prior_p(p_post)
    approx_h_list.append(ex_fact.compute_entropy())
    turns -=1
end = time.time()

approx_timecost = end - start
print("approx:{} s".format(approx_timecost))

approx:2.247291088104248 s


## random 算法

In [33]:
all_h_list = []
cost_list = []
post_p_list = []
import copy
for _ in range(100):
    random_fact_i = copy.deepcopy(random_fact)
    random_fact.set_prior_p(p_prior)
    turns_r=total_turns
    random_h_list=[random_fact.compute_entropy()]
    random_selection_idxes, _, _  =  random_selector.select(random_fact, budget, acc)
    while turns_r>0:
      
        if api_use:
            ans_r = [1 if gpt_check(ix_r, c_set)=="yes" else 0 for ix_r in random_selection_idxes]
        else:
            ans_r = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in random_selection_idxes]
        p_a_r,p_a_v_r = random_fact.compute_ans_p(ans_r, random_selection_idxes, acc)
        p_post = random_fact.get_prior_p()*p_a_v / p_a
        random_fact.set_prior_p(p_post)
        random_h_list.append(random_fact.compute_entropy())
        turns_r -=1
    all_h_list.append(random_h_list)
    cost_list.append(cost_sum_r)

NameError: name 'p_prior' is not defined

## Brute Algorithm

In [ ]:
import numpy as np
import time

brute_h_list=[brute_fact.compute_entropy()]
cost_sum = 0
turns = total_turns 
start = time.time()
while turns>0:
    selection_idxes, sub_facts, h = base_selector.select(brute_fact, budget, acc)
    if api_use:
         ans = [1 if gpt_check(ix_r, c_set)=="yes" else 0 for ix_r in selection_idxes]
    else:
        ans = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in selection_idxes]
    p_a,p_a_v = brute_fact.compute_ans_p(ans, selection_idxes, acc)
    p_post = brute_fact.get_prior_p()*p_a_v / p_a
    brute_fact.set_prior_p(p_post)
    brute_h_list.append(brute_fact.compute_entropy())
    turns -= 1
end = time.time()

brute_timecost = end - start
print("brute:{} s".format(brute_timecost))

brute:1.3460352420806885 s


## Heuristic 算法

In [ ]:
# heuristic_h_list=[heuristic_fact.compute_entropy()]
# cost_sum = 0
# turns = total_turns 
# start = time.time()
# while turns>0:
#     selection_idxes, sub_facts, h = h_selector.select(heuristic_fact, budget, acc, max_iters=10, cost_func=2)
#     if api_use:
#          ans = [1 if gpt_check(ix_r, c_set)=="yes" else 0 for ix_r in selection_idxes]
#     else:
#         ans = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in selection_idxes]
#     p_a,p_a_v = heuristic_fact.compute_ans_p(ans, selection_idxes, acc)
#     p_post = heuristic_fact.get_prior_p()*p_a_v / p_a
#     heuristic_fact.set_prior_p(p_post)
#     heuristic_h_list.append(heuristic_fact.compute_entropy())
#     turns -=1
# end = time.time()

# heuristic_timecost = end - start
# print("heuristic: {}".format(heuristic_timecost))

approx_entropies

In [ ]:
approx_h_list

[2.0179569806623436,
 0.0726139700924392,
 0.0009764466376386533,
 1.1357110128992981e-05,
 2.6316264512417873e-07,
 1.0962643634650122e-08,
 2.5179382450626204e-10,
 1.055894774481144e-11,
 2.345159380883632e-13,
 9.652416814242332e-15,
 2.0487819119090236e-16,
 8.377401183747016e-18,
 1.786662880923442e-19,
 7.23917548541768e-21,
 1.5232095493309028e-22,
 6.129705204406908e-24,
 1.276586254228878e-25,
 5.1101352712891224e-27,
 1.0556907607953479e-28,
 4.208111678938558e-30,
 8.636995214634737e-32]

random_entropies

In [ ]:
all_h_list

[[0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033],
 [0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033],
 [0.3662040962227033,
  0.3662040962227033,
  0.3662040962227033,
  0.3662

In [ ]:
import numpy as np
n = np.array(all_h_list)
random_h_l = n.mean(axis=0, keepdims=True)
random_h_l = random_h_l.tolist()[0]
random_h_l

[0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267,
 0.36620409622270267]

heuristic

In [ ]:
# heuristic_h_list

brute

In [ ]:
brute_h_list

[2.0179569806623436,
 0.0726139700924392,
 0.0009764466376386533,
 1.1357110128992981e-05,
 1.2090889087437317e-07,
 1.219222718258808e-09,
 1.1862304265033002e-11,
 1.1255795995197808e-13,
 1.0215233119514138e-15,
 9.399199047768724e-18,
 8.544428035770667e-20,
 7.691583012921329e-22,
 6.867892970193264e-24,
 6.090680807299e-26,
 5.370054776865246e-28,
 4.710950613981401e-30,
 4.114671145559478e-32,
 3.5800383224022343e-34,
 3.104247482820434e-36,
 2.683493853198695e-38,
 2.3134257122857743e-40]

In [ ]:
import json
dataset = "output"
def save_h_file(approx_name_f,random_f, brute_name_f, heuristic_name_f, approx_ob, random_ob, brute_ob, heuristic_ob):
    with open(f"./{dataset}/"+approx_name_f, "w") as f:
        json.dump(approx_ob, f, indent=2, ensure_ascii=False)
    
    with open(f"./{dataset}/"+random_f, "w") as w:
        json.dump(random_ob, w, indent=2, ensure_ascii=False)
        
    with open(f"./{dataset}/"+brute_name_f, "w") as f2:
        json.dump(brute_ob, f2, indent=2, ensure_ascii=False)
        
    with open(f"./{dataset}/"+ heuristic_name_f, "w") as f3:
        json.dump(heuristic_ob, f3, indent=2, ensure_ascii=False)

In [ ]:
approx = {"entropy":approx_h_list, "timecost":approx_timecost, "prob":list(ex_fact.get_prior_p())}
random = {"entropy":random_h_list, "timecost":0, "prob":list(random_fact.get_prior_p())}
brute = {"entropy":brute_h_list, "timecost":brute_timecost, "prob":list(brute_fact.get_prior_p())}
# heuristic = {"entropy":heuristic_h_list, "timecost":heuristic_timecost, "prob":list(heuristic_fact.get_prior_p())}

In [ ]:
name = f"turns_{total_turns}" + "_budget={}_{}"
save_h_file(name.format(budget,f"{dataset_name}_approx.json"), name.format(budget,f"{dataset_name}_random.json"),
            name.format(budget, f"{dataset_name}_heuristic.json"), name.format(budget, f"{dataset_name}_brute.json"),
            approx, random, heuristic, brute)

NameError: name 'heuristic' is not defined

In [ ]:
approx

{'entropy': [2.524036208602731,
  2.12485216899556,
  1.2997573169602883,
  0.2695479698312213,
  0.07517232246648327,
  0.024949054661128102,
  0.003463698461885738,
  0.00105368412312793,
  0.00031592810083154254,
  4.8198101523844665e-05,
  1.2468004286670111e-05,
  3.6069223723578727e-06,
  4.502870312009741e-07,
  1.2572119153165208e-07,
  3.678690757791902e-08,
  4.496091590832251e-09,
  1.26022525674622e-09,
  3.6449109569707313e-10,
  4.383476745223963e-11,
  1.2249805798276848e-11,
  3.5070807709703505e-12],
 'timecost': 1.593247890472412,
 'prob': [7.115321105261693e-19,
  7.115321105261693e-19,
  1.9375713667440716e-32,
  1.9375713667440716e-32,
  0.9999999999998914,
  2.4224079738243198e-15,
  2.7018651674951487e-14,
  6.545019725939145e-29,
  2.717999410571602e-14,
  6.584103445019168e-29,
  2.59537115612204e-14,
  6.287047783624355e-29,
  7.012342923448502e-28,
  1.6986755412954053e-42,
  7.054217272555002e-28,
  1.7088192170128338e-42,
  2.59537115612204e-14,
  6.2870477

In [ ]:
import time
start = time.time()
end = time.time()
end - start

In [ ]:
print( type(end-start))